In [103]:
import tensorflow as tf
import time
import numpy as np
import os
import copy
import pickle
import argparse
import utility
import pandas as pd
from sklearn.metrics import *


In [228]:

class DPR_REO:

    def __init__(self, sess, args, train_df, vali_df, item_genre, genre_error_weight
                 , key_genre, item_genre_list, user_genre_count):
        self.dataname = args.dataname

        self.layers = eval(args.layers)

        self.key_genre = key_genre
        self.item_genre_list = item_genre_list
        self.user_genre_count = user_genre_count

        self.sess = sess
        self.args = args

        self.num_cols = len(train_df['item_id'].unique())
        self.num_rows = len(train_df['user_id'].unique())

        self.hidden_neuron = args.hidden_neuron
        self.neg = args.neg
        self.batch_size = args.batch_size

        self.train_df = train_df
        self.vali_df = vali_df
        self.num_train = len(self.train_df)
        self.num_vali = len(self.vali_df)

        self.train_epoch = args.train_epoch
        self.train_epoch_a = args.train_epoch_a
        self.display_step = args.display_step

        self.lr_r = args.lr_r  # learning rate
        self.lr_a = args.lr_a  # learning rate

        self.reg = args.reg  # regularization term trade-off
        self.reg_s = args.reg_s

        self.num_genre = args.num_genre
        self.alpha = args.alpha
        self.item_genre = item_genre
        self.genre_error_weight = genre_error_weight

        self.genre_count_list = []
        for k in range(self.num_genre):
            self.genre_count_list.append(np.sum(item_genre[:, k]))

        print('**********DPR_REO**********')
        print(self.args)
        self._prepare_model()


    #load bpr checkpoint
    def loadmodel(self, saver, checkpoint_dir):
        ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
            saver.restore(self.sess, os.path.join(checkpoint_dir, ckpt_name))
            
            return True
        else:
            return False

    def run(self):
        init = tf.compat.v1.global_variables_initializer()
        self.sess.run(init)

        saver = tf.compat.v1.train.Saver([self.P, self.Q])
        self.loadmodel(saver, "./"+self.dataname+"/BPR_check_points")
        print ("restore done")

        for epoch_itr in range(1, self.train_epoch + 1 + self.train_epoch_a):
            print("starting itr " + str(epoch_itr))
            self.train_model(epoch_itr)
            if epoch_itr % self.display_step == 0:
                self.test_model(epoch_itr)
            print("end itr " + str(epoch_itr))
        return self.make_records()

    def _prepare_model(self):
        with tf.name_scope("input_data"):
            # declare embedding u i j, genre to predict
            self.user_input = tf.compat.v1.placeholder(tf.int32, shape=[None, 1], name="user_input")
            self.item_input_pos = tf.compat.v1.placeholder(tf.int32, shape=[None, 1], name="item_input_pos")
            self.item_input_neg = tf.compat.v1.placeholder(tf.int32, shape=[None, 1], name="item_input_neg")

            self.input_item_genre = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, self.num_genre]
                                                   , name="input_item_genre")
            self.input_item_error_weight = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, 1]
                                                          , name="input_item_error_weight")
        #this is the part initialize BPR for MF
        with tf.compat.v1.variable_scope("BPR", reuse=tf.compat.v1.AUTO_REUSE):
            self.P = tf.compat.v1.get_variable(name="P",
                                     initializer=tf.compat.v1.truncated_normal(shape=[self.num_rows, self.hidden_neuron], mean=0,
                                                                     stddev=0.03), dtype=tf.float32)
            self.Q = tf.compat.v1.get_variable(name="Q",
                                     initializer=tf.compat.v1.truncated_normal(shape=[self.num_cols, self.hidden_neuron], mean=0,
                                                                     stddev=0.03), dtype=tf.float32)
        para_r = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope="BPR")


        #adversary as a mlp
        with tf.compat.v1.variable_scope("Adversarial", reuse=tf.compat.v1.AUTO_REUSE):
            num_layer = len(self.layers)
            adv_W = []
            adv_b = []
            for l in range(num_layer):
                if l == 0:
                    in_shape = 1
                else:
                    in_shape = self.layers[l - 1]
                adv_W.append(tf.compat.v1.get_variable(name="adv_W" + str(l),
                                             initializer=tf.compat.v1.truncated_normal(shape=[in_shape, self.layers[l]],
                                                                             mean=0, stddev=0.03), dtype=tf.float32))
                adv_b.append(tf.compat.v1.get_variable(name="adv_b" + str(l),
                                             initializer=tf.compat.v1.truncated_normal(shape=[1, self.layers[l]],
                                                                             mean=0, stddev=0.03), dtype=tf.float32))
            adv_W_out = tf.compat.v1.get_variable(name="adv_W_out",
                                        initializer=tf.compat.v1.truncated_normal(shape=[self.layers[-1], self.num_genre],
                                                                        mean=0, stddev=0.03), dtype=tf.float32)

            adv_b_out = tf.compat.v1.get_variable(name="adv_b_out",
                                        initializer=tf.compat.v1.truncated_normal(shape=[1, self.num_genre],
                                                                        mean=0, stddev=0.03), dtype=tf.float32)
        para_a = tf.compat.v1.get_collection(tf.compat.v1.GraphKeys.GLOBAL_VARIABLES, scope="Adversarial")


        ##prep embedding for bpr
        p = tf.reduce_sum(tf.nn.embedding_lookup(self.P, self.user_input), 1)
        self.p = p
        q_neg = tf.reduce_sum(tf.nn.embedding_lookup(self.Q, self.item_input_neg), 1)
        self.q_neg = q_neg
        q_pos = tf.reduce_sum(tf.nn.embedding_lookup(self.Q, self.item_input_pos), 1)
        self.q_pos = q_pos
        ##prediction based on bpr embedding
        predict_pos = tf.reduce_sum(p * q_pos, 1)
        self.predict_pos = predict_pos
        predict_neg = tf.reduce_sum(p * q_neg, 1)
        self.predict_neg = predict_neg
        r_cost1 = tf.reduce_sum(tf.nn.softplus(-(predict_pos - predict_neg))) #bpr cost
        r_cost2 = self.reg * 0.5 * (self.l2_norm(self.P) + self.l2_norm(self.Q))  # regularization term
        pred = tf.matmul(self.P, tf.transpose(self.Q))
        self.s_mean = tf.reduce_mean(pred, axis=1)
        self.s_std = tf.keras.backend.std(pred, axis=1)
        self.s_cost = tf.reduce_sum(tf.square(self.s_mean) + tf.square(self.s_std) - 2 * tf.compat.v1.log(self.s_std) - 1)
        self.r_cost = r_cost1 + r_cost2 + self.reg_s * 0.5 * self.s_cost


        #last layer = prediction by the mlp adversary
        adv_last = tf.reshape(predict_pos, [tf.shape(self.input_item_genre)[0], 1])
#         adv_last = tf.reshape(tf.concat([predict_pos, predict_neg], axis=0), [tf.shape(self.input_item_genre)[0], 1])
        self.adv_last_prem = adv_last
        for l in range(num_layer):
            adv = tf.nn.relu(tf.matmul(adv_last, adv_W[l]) + adv_b[l])
            adv_last = adv #linear-relu
            self.adv_last = adv_last
        #sigmoid to convert output to prediction
        self.adv_output = tf.nn.sigmoid(tf.matmul(adv_last, adv_W_out) + adv_b_out)

        #adversarial cost (why they have the input_item_error_weight??)
        self.a_cost = tf.reduce_sum(tf.square(self.adv_output - self.input_item_genre) * self.input_item_error_weight)

        self.all_cost = self.r_cost - self.alpha * self.a_cost  # the loss function

        #optimize for bpr, adversary and overall cost wrt to overall obj function
        with tf.compat.v1.variable_scope("Optimizer", reuse=tf.compat.v1.AUTO_REUSE):
            self.r_optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.lr_r).minimize(self.r_cost, var_list=para_r)
            self.a_optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.lr_a).minimize(self.a_cost, var_list=para_a)
            self.all_optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=self.lr_r).minimize(self.all_cost, var_list=para_r)

    def train_model(self, itr):
        NS_start_time = time.time() * 1000.0
        epoch_r_cost = 0.0
        epoch_s_cost = 0.0
        epoch_s_mean = 0.0
        epoch_s_std = 0.0
        epoch_a_cost = 0.0
        num_sample, user_list, item_pos_list, item_neg_list = utility.negative_sample(self.train_df, self.num_rows,
                                                                                      self.num_cols, self.neg)
        NS_end_time = time.time() * 1000.0

        start_time = time.time() * 1000.0
        num_batch = int(num_sample / float(self.batch_size)) + 1
        random_idx = np.random.permutation(num_sample)
        for i in range(num_batch):
            # get the indices of the current batch
            if i == num_batch - 1:
                batch_idx = random_idx[i * self.batch_size:]
            elif i < num_batch - 1:
                batch_idx = random_idx[(i * self.batch_size):((i + 1) * self.batch_size)]

            if itr > self.train_epoch:
                random_idx_a = np.random.permutation(num_sample)
                print("start itr " + str(itr) + "for batch " + str(i))
                for j in range(num_batch):
                    if j == num_batch - 1:
                        batch_idx_a = random_idx_a[j * self.batch_size:]
                    elif j < num_batch - 1:
                        batch_idx_a = random_idx_a[(j * self.batch_size):((j + 1) * self.batch_size)]
                    item_idx_list = ((item_pos_list[batch_idx_a, :]).reshape((len(batch_idx_a)))).tolist()
                    a_opt, tmp_a_cost, p, q_pos,q_neg, predict_pos, adv_last_prem,adv_last, adv_output = self.sess.run(  # do the optimization by the minibatch
                        [self.a_optimizer, self.a_cost, self.p, self.q_pos,self.q_neg, self.predict_pos, self.adv_last_prem,self.adv_last,self.adv_output ], #update the adversary
                        feed_dict={self.user_input: user_list[batch_idx_a, :],
                                   self.item_input_pos: item_pos_list[batch_idx_a, :],
                                   self.item_input_neg: item_neg_list[batch_idx_a, :],
                                   self.input_item_genre: self.item_genre[item_idx_list, :],
                                   self.input_item_error_weight: self.genre_error_weight[item_idx_list, :]})
                    epoch_a_cost += tmp_a_cost
                    print("update epoch_a_cost done " + str(j))
                    print("Shape of p = " + str(p.shape) +" sample of an embedding")
                    print(p[1,:])
                    print("Shape of q_pos = " + str(q_pos.shape)+" sample of an embedding")
                    print(q_pos[1,:])
                    print("Shape of predict_pos = " + str(predict_pos.shape)+" sample of some predictions")
                    print(predict_pos[1:5])
                    print("Shape of input to adversary = " + str(adv_last_prem.shape) +" sample of an input")
                    print(adv_last_prem[1:5,:])
#                     print("Shape of adv_activated = " + str(adv_last.shape))
                    print("Shape of adv_output = " + str(adv_output.shape) +" sample of an output")
                    print(adv_output[1:5,:])
                item_idx_list = ((item_pos_list[batch_idx, :]).reshape((len(batch_idx)))).tolist()
                _, tmp_r_cost, tmp_s_cost, tmp_s_mean, tmp_s_std = self.sess.run(  # do the optimization by the minibatch
                    [self.all_optimizer, self.all_cost, self.s_cost, self.s_mean, self.s_std], # update overall objective
                    feed_dict={self.user_input: user_list[batch_idx, :],
                               self.item_input_pos: item_pos_list[batch_idx, :],
                               self.item_input_neg: item_neg_list[batch_idx, :],
                               self.input_item_genre: self.item_genre[item_idx_list, :],
                               self.input_item_error_weight: self.genre_error_weight[item_idx_list, :]})
                epoch_r_cost += tmp_r_cost
                epoch_s_mean += np.mean(tmp_s_mean)
                epoch_s_std += np.mean(tmp_s_std)
                epoch_s_cost += tmp_s_cost
                print("update epoch_s_cost done")
            else:#for train_epoch, we only train the classifer (BPR) then append the adversarial training phase
                item_idx_list = ((item_pos_list[batch_idx, :]).reshape((len(batch_idx)))).tolist()
                _, tmp_r_cost, tmp_s_cost, tmp_s_mean, tmp_s_std = self.sess.run(  # do the optimization by the minibatch
                    [self.r_optimizer, self.r_cost, self.s_cost, self.s_mean, self.s_std], #update the weight of the classifier
                    feed_dict={self.user_input: user_list[batch_idx, :],
                               self.item_input_pos: item_pos_list[batch_idx, :],
                               self.item_input_neg: item_neg_list[batch_idx, :],
                               self.input_item_genre: self.item_genre[item_idx_list, :],
                               self.input_item_error_weight: self.genre_error_weight[item_idx_list, :]})
                epoch_r_cost += tmp_r_cost
                epoch_s_mean += np.mean(tmp_s_mean)
                epoch_s_std += np.mean(tmp_s_std)
                epoch_s_cost += tmp_s_cost
        print("adjust a_cost")                  
        epoch_a_cost /= num_batch
        if itr % self.display_step == 0:
            print ("Training //", "Epoch %d //" % itr, " Total r_cost = %.5f" % epoch_r_cost,
                   " Total s_cost = %.5f" % epoch_s_cost,
                   " Total s_mean = %.5f" % epoch_s_mean,
                   " Total s_std = %.5f" % epoch_s_std,
                   " Total a_cost = %.5f" % epoch_a_cost,
                   "Training time : %d ms" % (time.time() * 1000.0 - start_time),
                   "negative Sampling time : %d ms" % (NS_end_time - NS_start_time),
                   "negative samples : %d" % (num_sample))

    def test_model(self, itr):  # calculate the cost and rmse of testing set in each epoch
        if itr % self.display_step == 0:
            start_time = time.time() * 1000.0
            P, Q = self.sess.run([self.P, self.Q])
            Rec = np.matmul(P, Q.T)

            [precision, recall, f_score, NDCG] = utility.test_model_all(Rec, self.vali_df, self.train_df)
            utility.ranking_analysis(Rec, self.vali_df, self.train_df, self.key_genre, self.item_genre_list,
                                     self.user_genre_count)
            print("testing computation done")
            avg_genre = []
            for k in range(self.num_genre):
                avg_genre.append(np.sum(Rec * self.item_genre[:, k].reshape((1, self.num_cols)))
                                 / (self.genre_count_list[k] * self.num_rows))

            std_avg_genre = np.std(avg_genre)
            print (
                "Testing //", "Epoch %d //" % itr,
                "Testing time : %d ms" % (time.time() * 1000.0 - start_time))
            print("avg rating genre: " + str(avg_genre))
            print("std of avg rating genre" + str(std_avg_genre))
            print("=" * 200)

    def make_records(self):  # record all the results' details into files
        P, Q = self.sess.run([self.P, self.Q])
        Rec = np.matmul(P, Q.T)

        [precision, recall, f_score, NDCG] = utility.test_model_all(Rec, self.vali_df, self.train_df)
        return precision, recall, f_score, NDCG, Rec

    @staticmethod
    def l2_norm(tensor):
        return tf.reduce_sum(tf.square(tensor))



In [229]:
parser = argparse.ArgumentParser(description='DPR_REO')
parser.add_argument('-f')
parser.add_argument('--train_epoch', type=int, default=0)
parser.add_argument('--train_epoch_a', type=int, default=20)
parser.add_argument('--display_step', type=int, default=1)
parser.add_argument('--lr_r', type=float, default=0.01)
parser.add_argument('--lr_a', type=float, default=0.005)
parser.add_argument('--reg', type=float, default=0.1)
parser.add_argument('--reg_s', type=float, default=30)
parser.add_argument('--hidden_neuron', type=int, default=20)
parser.add_argument('--n', type=int, default=1)
parser.add_argument('--neg', type=int, default=5)
parser.add_argument('--alpha', type=float, default=1000.0)
parser.add_argument('--batch_size', type=int, default=1024)
parser.add_argument('--layers', nargs='?', default='[50, 50, 50, 50]')
parser.add_argument('--dataname', nargs='?', default='ml1m-6')
args = parser.parse_args()

In [230]:
train_df = pd.read_pickle(r'ml1m-6/training_df.pkl')
vali_df = pd.read_pickle(r'ml1m-6/valiing_df.pkl')   # for validation
# vali_df = pickle.load(open('./' + dataname + '/testing_df.pkl'))  # for testing
key_genre = pd.read_pickle(r'ml1m-6/key_genre.pkl')
item_idd_genre_list = pd.read_pickle(r'ml1m-6/item_idd_genre_list.pkl')
genre_item_vector = pd.read_pickle(r'ml1m-6/genre_item_vector.pkl')
genre_count = pd.read_pickle(r'ml1m-6/genre_count.pkl')
user_genre_count = pd.read_pickle(r'ml1m-6/user_genre_count.pkl')

In [231]:

num_item = len(train_df['item_id'].unique())
num_user = len(train_df['user_id'].unique())
num_genre = len(key_genre)

item_genre_list = []
for u in range(num_item):
    gl = item_idd_genre_list[u]
    tmp = []
    for g in gl:
        if g in key_genre:
            tmp.append(g)
    item_genre_list.append(tmp)

print('!' * 100)
print('number of positive feedback: ' + str(len(train_df)))
print('estimated number of training samples: ' + str(args.neg * len(train_df)))
print('!' * 100)

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
number of positive feedback: 370229
estimated number of training samples: 1851145
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!


In [232]:
item_genre = np.zeros((num_item, num_genre))
for i in range(num_item):
    gl = item_genre_list[i]
    for k in range(num_genre):
        if key_genre[k] in gl:
            item_genre[i, k] = 1.0

genre_count_mean_reciprocal = []

##there are six key_genre --> in the training dataset, count the number of movies for each genre
#genre_count = dictionary with number of movies for each keygrenre
for k in key_genre:
    genre_count_mean_reciprocal.append(1.0 / genre_count[k])
genre_count_mean_reciprocal = (np.array(genre_count_mean_reciprocal)).reshape((num_genre, 1))
genre_error_weight = np.dot(item_genre, genre_count_mean_reciprocal)

args.num_genre = num_genre


In [233]:
precision = np.zeros(4)
recall = np.zeros(4)
f1 = np.zeros(4)
ndcg = np.zeros(4)
RSP = np.zeros(4)
REO = np.zeros(4)

# tf.compat.v1.disable_eager_execution()

n = args.n
for i in range(n):
    with tf.compat.v1.Session() as sess:
        dpr = DPR_REO(sess, args, train_df, vali_df, item_genre, genre_error_weight,
                      key_genre, item_genre_list, user_genre_count)
        [prec_one, rec_one, f_one, ndcg_one, Rec] = dpr.run()
        [RSP_one, REO_one] = utility.ranking_analysis(Rec, vali_df, train_df, key_genre, item_genre_list,
                                                      user_genre_count)
        precision += prec_one
        recall += rec_one
        f1 += f_one
        ndcg += ndcg_one
        RSP += RSP_one
        REO += REO_one

with open('Rec_' + dataname + '_DPR_REO.mat', "wb") as f:
    np.save(f, Rec)

**********DPR_REO**********
Namespace(f='/home/vuhoang181/.local/share/jupyter/runtime/kernel-2aaa66f4-45f7-4b84-8d88-9ed21cb2aec6.json', train_epoch=0, train_epoch_a=20, display_step=1, lr_r=0.01, lr_a=0.005, reg=0.1, reg_s=30, hidden_neuron=20, n=1, neg=5, alpha=1000.0, batch_size=1024, layers='[50, 50, 50, 50]', dataname='ml1m-6', num_genre=6)
INFO:tensorflow:Restoring parameters from ./ml1m-6/BPR_check_points/check_point.ckpt-30
restore done
starting itr 1
start itr 1for batch 0
update epoch_a_cost done 0
Shape of p = (1024, 20) sample of an embedding
[ 0.74132526  0.08147587  0.19108857  0.83912146 -0.43124998 -0.21937485
  0.11078015  0.7932653  -0.09588693 -0.2380501  -0.5485486   0.22993924
 -0.5348857  -0.06496265 -0.08671466  0.02010908  1.058053    1.1864057
 -1.3438265   0.02202923]
Shape of q_pos = (1024, 20) sample of an embedding
[-0.28453866 -0.59588885 -0.9265488   0.63192403  0.44817594  0.30437788
 -1.5870682  -1.483597    0.20383397  0.09430871 -1.6786795  -0.247112

update epoch_a_cost done 36
Shape of p = (1024, 20) sample of an embedding
[ 0.02467574 -0.47065884 -0.5251937   0.8200332   0.5080045   0.17859402
 -0.12080427 -0.5572282  -0.63883764 -0.59712416 -0.37266618  0.3256548
 -0.02674643  0.12119216 -0.8955604   0.37471625  0.08593425  0.10879597
 -0.14134718  0.5841689 ]
Shape of q_pos = (1024, 20) sample of an embedding
[-0.16139784  0.35666057 -0.74618596  1.5103012   0.08732244  0.9110126
 -0.394949    0.7179354  -0.64912856  0.357473    0.64044094  0.5167116
 -0.38940436  0.36707863 -0.35114926  0.32635555  0.9417067   0.16617957
 -0.04734379  0.48674256]
Shape of predict_pos = (1024,) sample of some predictions
[2.3257961 4.1307044 3.4047267 4.7086678]
Shape of input to adversary = (1024, 1) sample of an input
[[2.3257961]
 [4.1307044]
 [3.4047267]
 [4.7086678]]
Shape of adv_output = (1024, 6) sample of an output
[[0.4296445  0.14353205 0.2069395  0.40055838 0.16247585 0.20008923]
 [0.42047265 0.10971633 0.17467885 0.38551235 0.128221

update epoch_a_cost done 84
Shape of p = (1024, 20) sample of an embedding
[ 0.7000837   0.509211   -0.41378832  0.8135664  -1.2304765   0.7118238
 -0.17419021  0.13806304 -0.18205856  0.44198123  0.03394911 -0.13100989
  0.09733929  0.80694616 -0.44090825 -0.04411967  0.5439503   0.01147673
 -0.39081895 -0.41264808]
Shape of q_pos = (1024, 20) sample of an embedding
[ 1.1305245   0.6395598   1.7636721   0.50141776 -0.04668585  0.819749
  0.5201726   0.37567982 -0.17772613  0.97935426  0.11181846 -0.7079843
  0.243982    1.0675672  -0.04361732  0.05102282  0.704085    0.9673902
  0.15803029 -0.5305345 ]
Shape of predict_pos = (1024,) sample of some predictions
[3.4127162 4.717527  3.44319   1.8125335]
Shape of input to adversary = (1024, 1) sample of an input
[[3.4127162]
 [4.717527 ]
 [3.44319  ]
 [1.8125335]]
Shape of adv_output = (1024, 6) sample of an output
[[0.3849918  0.14497967 0.2125402  0.36403847 0.16982529 0.19846158]
 [0.38251904 0.13923572 0.20727108 0.36088446 0.16403265

update epoch_a_cost done 158
Shape of p = (1024, 20) sample of an embedding
[ 0.34649795  0.02788384 -0.5319755   0.506412   -0.3534272  -0.1531203
  0.1389356  -0.39027444 -0.490889    0.5634618  -0.5771713  -0.18618178
  0.00113998  0.33865845  0.51496494 -0.07710862  0.84408563 -0.45263526
 -0.19049329  0.0482631 ]
Shape of q_pos = (1024, 20) sample of an embedding
[-0.7915846   0.7987339  -1.5199684  -0.3035801   0.4903153   0.99918103
 -1.0448229   0.22194088 -0.97803015  0.82487583 -0.53496605  0.05785109
 -0.7091342   1.3048483   0.51972157  0.2996907   0.8543435   0.1545324
  0.52304405  0.36415344]
Shape of predict_pos = (1024,) sample of some predictions
[2.3424087 5.1410155 1.343275  3.6966429]
Shape of input to adversary = (1024, 1) sample of an input
[[2.3424087]
 [5.1410155]
 [1.343275 ]
 [3.6966429]]
Shape of adv_output = (1024, 6) sample of an output
[[0.40422872 0.14723301 0.21515313 0.37350634 0.16447683 0.19966957]
 [0.39919838 0.1321437  0.20126677 0.36581567 0.1492

update epoch_a_cost done 210
Shape of p = (1024, 20) sample of an embedding
[ 0.2289202   0.9370873  -0.4704043   0.04816499 -0.24676664 -0.003064
 -0.304827   -0.7038712   0.03485391 -0.3156766  -0.4142221  -0.38581732
 -0.5605766  -0.24581234  1.5911064  -0.29813617  0.93117523  0.24830927
 -0.30312642  0.52513   ]
Shape of q_pos = (1024, 20) sample of an embedding
[-0.6581656   0.95128286 -0.18947975 -0.3996482  -1.0181515   0.5741648
 -1.012233   -1.9013382  -0.7980744   0.99229497  0.30927384 -0.5402679
 -0.03640322  0.22147325  1.6389557  -0.4629588   0.37394005 -0.2973969
 -1.4025664   0.40561023]
Shape of predict_pos = (1024,) sample of some predictions
[6.070521   0.03131673 2.5763383  4.9530854 ]
Shape of input to adversary = (1024, 1) sample of an input
[[6.070521  ]
 [0.03131673]
 [2.5763383 ]
 [4.9530854 ]]
Shape of adv_output = (1024, 6) sample of an output
[[0.39295644 0.13272959 0.18127702 0.38049972 0.16772448 0.18700388]
 [0.39860725 0.15027194 0.19760783 0.388157   0

...

update epoch_a_cost done 474
Shape of p = (1024, 20) sample of an embedding
[ 1.0012614   0.09188645  0.20784706  0.28943306 -0.27647743  0.41888767
  0.04415692  0.18581836 -1.7651287   1.2797794  -0.68015695  1.172469
 -0.5460518   0.24967673  0.33809572  0.3269778   0.4486603  -0.77155554
 -0.37595478  0.0150261 ]
Shape of q_pos = (1024, 20) sample of an embedding
[ 0.18367398 -0.04232401 -1.5311403   0.55715466 -1.0197786  -0.12442766
 -0.6438394  -1.1820662   0.23725328  1.1262316  -0.44496578  0.44469368
 -0.04296285  0.15175746 -0.00581714 -2.1043932  -0.8648821   0.24910493
 -0.9037299  -0.10142046]
Shape of predict_pos = (1024,) sample of some predictions
[0.98065376 5.6331415  5.259348   2.1936903 ]
Shape of input to adversary = (1024, 1) sample of an input
[[0.98065376]
 [5.6331415 ]
 [5.259348  ]
 [2.1936903 ]]
Shape of adv_output = (1024, 6) sample of an output
[[0.27838567 0.21779867 0.2092682  0.3066985  0.25021473 0.22110271]
 [0.5174126  0.09237216 0.19678342 0.4265777

update epoch_a_cost done 505
Shape of p = (1024, 20) sample of an embedding
[ 0.19998132 -0.2882828  -0.19124453  0.19877562 -0.21212615 -0.38335797
  0.15333359  0.4394615  -0.3001561   0.9296581   0.42309004  0.5724323
 -0.02719022  0.14715315  0.3284539  -0.44711444  0.76117766  1.7560716
 -0.34086072  0.19263844]
Shape of q_pos = (1024, 20) sample of an embedding
[-0.302789    0.6654931   0.9765463   1.6026179  -0.9345766  -0.0187317
  0.3036641   0.49937877 -0.80554855  0.7297898  -0.759369    0.322207
  0.00732519  0.43245155  0.3486302   0.37610108  0.12713222  0.7340147
 -0.83242816  0.42369205]
Shape of predict_pos = (1024,) sample of some predictions
[2.8951542 5.030857  4.164642  6.766246 ]
Shape of input to adversary = (1024, 1) sample of an input
[[2.8951542]
 [5.030857 ]
 [4.164642 ]
 [6.766246 ]]
Shape of adv_output = (1024, 6) sample of an output
[[0.36418018 0.14686075 0.2058502  0.37563378 0.17237155 0.205301  ]
 [0.46021935 0.1106296  0.2031602  0.42567953 0.13310014

update epoch_a_cost done 534
Shape of p = (1024, 20) sample of an embedding
[-0.4768884  -0.15276344 -0.27206424  0.47814307 -0.554963    0.34270275
 -0.5023021  -0.13119873 -0.49344203  0.48513192  0.5031453   0.36882624
 -0.3821904   1.5273259  -0.20256764  0.2189239  -0.3922445   0.07983452
 -0.9311476   0.7165386 ]
Shape of q_pos = (1024, 20) sample of an embedding
[ 1.1765591  -0.8389883  -1.5245911   0.2795838  -0.2604534  -0.44479293
 -0.61443317 -0.39100152 -0.89219546  1.5229923   0.5951826  -0.2558344
  0.6433443   0.50042254 -0.79172486  0.4362398   0.1586389  -0.9706392
 -0.2930635  -0.35764796]
Shape of predict_pos = (1024,) sample of some predictions
[2.5030017 3.508538  3.648959  6.544635 ]
Shape of input to adversary = (1024, 1) sample of an input
[[2.5030017]
 [3.508538 ]
 [3.648959 ]
 [6.544635 ]]
Shape of adv_output = (1024, 6) sample of an output
[[0.3310384  0.16769902 0.20682283 0.33304647 0.20233434 0.194081  ]
 [0.36930856 0.15486436 0.21673304 0.3555682  0.1842

update epoch_a_cost done 562
Shape of p = (1024, 20) sample of an embedding
[ 0.01083999  0.20995286  0.9652249   1.1651723  -0.13862179  0.27279893
  0.44272703 -0.9408978   0.0655336   0.11365268 -1.3666937  -0.4409954
  0.07820836  1.0079792   0.21303901  0.10145626 -0.4847743   1.3182136
  0.01662506 -0.33825213]
Shape of q_pos = (1024, 20) sample of an embedding
[-0.05465484  0.8218548   1.6272222   0.6826114  -0.10142067  0.31184933
  0.8633248   0.70351875 -1.0032773   0.8588172  -0.17036709  0.5551708
  0.3914546   0.03774087  0.09059743  0.17135796  0.85676455  1.5285326
 -0.05568609  0.58422774]
Shape of predict_pos = (1024,) sample of some predictions
[3.8836296 3.6373837 5.409251  6.7507887]
Shape of input to adversary = (1024, 1) sample of an input
[[3.8836296]
 [3.6373837]
 [5.409251 ]
 [6.7507887]]
Shape of adv_output = (1024, 6) sample of an output
[[0.37250194 0.15120102 0.20296237 0.3593126  0.1730569  0.19220746]
 [0.36317548 0.15482928 0.20261289 0.35402718 0.176764

KeyboardInterrupt: 

In [149]:

precision /= n
recall /= n
f1 /= n
ndcg /= n
RSP /= n
REO /= n

print('')
print('*' * 100)
print('Averaged precision@1\t%.7f,\t||\tprecision@5\t%.7f,\t||\tprecision@10\t%.7f,\t||\tprecision@15\t%.7f' \
      % (precision[0], precision[1], precision[2], precision[3]))
print('Averaged recall@1\t%.7f,\t||\trecall@5\t%.7f,\t||\trecall@10\t%.7f,\t||\trecall@15\t%.7f' \
      % (recall[0], recall[1], recall[2], recall[3]))
print('Averaged f1@1\t\t%.7f,\t||\tf1@5\t\t%.7f,\t||\tf1@10\t\t%.7f,\t||\tf1@15\t\t%.7f' \
      % (f1[0], f1[1], f1[2], f1[3]))
print('Averaged NDCG@1\t\t%.7f,\t||\tNDCG@5\t\t%.7f,\t||\tNDCG@10\t\t%.7f,\t||\tNDCG@15\t\t%.7f' \
      % (ndcg[0], ndcg[1], ndcg[2], ndcg[3]))
print('*' * 100)
print('Averaged RSP    @1\t%.7f\t||\t@5\t%.7f\t||\t@10\t%.7f\t||\t@15\t%.7f' \
      % (RSP[0], RSP[1], RSP[2], RSP[3]))
print('Averaged REO @1\t%.7f\t||\t@5\t%.7f\t||\t@10\t%.7f\t||\t@15\t%.7f' \
      % (REO[0], REO[1], REO[2], REO[3]))
print('*' * 100)


****************************************************************************************************
Averaged precision@1	0.0000000,	||	precision@5	0.0000000,	||	precision@10	0.0000000,	||	precision@15	0.0000000
Averaged recall@1	0.0000000,	||	recall@5	0.0000000,	||	recall@10	0.0000000,	||	recall@15	0.0000000
Averaged f1@1		0.0000000,	||	f1@5		0.0000000,	||	f1@10		0.0000000,	||	f1@15		0.0000000
Averaged NDCG@1		0.0000000,	||	NDCG@5		0.0000000,	||	NDCG@10		0.0000000,	||	NDCG@15		0.0000000
****************************************************************************************************
Averaged RSP    @1	0.0000000	||	@5	0.0000000	||	@10	0.0000000	||	@15	0.0000000
Averaged REO @1	0.0000000	||	@5	0.0000000	||	@10	0.0000000	||	@15	0.0000000
****************************************************************************************************


In [27]:
10 % 1

0

In [111]:
num_cols = len(train_df['item_id'].unique())
num_rows = len(train_df['user_id'].unique())

In [119]:
P1 =tf.compat.v1.get_variable(name="P1",
                                     initializer=tf.compat.v1.truncated_normal(shape=[num_rows, 20], mean=0,
                                                                               stddev=0.03), dtype=tf.float32)


In [127]:
Q =tf.compat.v1.get_variable(name="Q",
                                     initializer=tf.compat.v1.truncated_normal(shape=[num_cols, 20], mean=0,
                                                                               stddev=0.03), dtype=tf.float32)

In [123]:
user_input=tf.compat.v1.placeholder(tf.int32, shape=[None, 1], name="user_input")

In [124]:
item_input_pos = tf.compat.v1.placeholder(tf.int32, shape=[None, 1], name="item_input_pos")

In [133]:
input_item_genre = tf.compat.v1.placeholder(dtype=tf.float32, shape=[None, 6]
                                                   , name="input_item_genre")

In [125]:
p = tf.reduce_sum(tf.nn.embedding_lookup(P1, user_input), 1)

In [128]:
q_pos = tf.reduce_sum(tf.nn.embedding_lookup(Q, item_input_pos), 1)

In [129]:
p

<tf.Tensor 'Sum_175:0' shape=(None, 20) dtype=float32>

In [130]:
q_pos

<tf.Tensor 'Sum_176:0' shape=(None, 20) dtype=float32>

In [131]:
predict_pos = tf.reduce_sum(p * q_pos, 1)

In [132]:
predict_pos

<tf.Tensor 'Sum_177:0' shape=(None,) dtype=float32>

In [134]:
adv_last = tf.reshape(predict_pos, [tf.shape(input_item_genre)[0], 1])

In [135]:
adv_last

<tf.Tensor 'Reshape_18:0' shape=(None, 1) dtype=float32>